In [1]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [2]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130
     |████████████████████████████████| 84 kB 213 kB/s             
     |████████████████████████████████| 748.8 MB 6.0 kB/s              
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [71]:
!rm -rf outputs

In [72]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/train_final_june23_all.csv")
len(df)

44397

In [73]:
df.source.unique()

array(['acl_cambridge', 'predicts_acl_pm2500_vocabed',
       'Oct1_clinical_studies_pm', 'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522', 'june152022_labels_envgeosoc',
       'june152022_labels_matscichem'], dtype=object)

In [74]:
df.fields.unique()

array(['Computer Science', 'Medicine', nan, 'Biology', 'Psychology',
       'Sociology', 'Environmental Science', 'Geology',
       'Materials Science'], dtype=object)

In [75]:
#downsample medical data in training set
# df = df[df["source"].notin(["oct3_labels", "june152022_labels_matscichem", 
#                            "june152022_labels_envgeosoc", "labels_oct7", 
#                            "psych1k_100522", "biology1k_050522", "predicts_acl_pm2500_vocabed", 
#                            'pm_CONCLUSIONS', 'biology1k_050522'])]

In [76]:
df[df.source =="biology1k_050522"]

,text,doi,journalName,title,fields,source,labels
40897,Osteogenesis imperfecta (OI) is a genetic bone...,10.1002/jbmr.3424,Journal of Bone and Mineral Research,Mutations That Alter the Carboxy‐Terminal‐Prop...,Biology,biology1k_050522,1
40898,Highly pathogenic avian influenza virus (HPAIV...,10.1371/journal.pone.0044447,PLoS ONE,Protective Efficacy of Newcastle Disease Virus...,Biology,biology1k_050522,1
40899,Central giant cell granuloma (CGCG) is a react...,10.1111/J.1600-0714.1999.TB01996.X,Journal of oral pathology & medicine : officia...,"Immunohistochemical expression of p53, MDM2, K...",Biology,biology1k_050522,1
40900,Fasciola hepatica infection continues to be a ...,10.1016/j.vetpar.2018.05.008,Veterinary parasitology,Intranasal delivery of a formulation containin...,Biology,biology1k_050522,1
40901,"O cravo-da-india, Syzygium aromaticum [L] Merr.",10.5935/RVQ.V6I5.715,NaN,Quantificação e Caracterização dos Principais ...,Biology,biology1k_050522,0
...,...,...,...,...,...,...,...
41916,"Bacterial diseases, mainly those caused by bac...",NaN,Boletim Do Instituto De Pesca,Uso de hidróxido de cálcio no controle de vibr...,Biology,biology1k_050522,1
41917,Staphylococcus aureus is a common facultative ...,NaN,NaN,Antimicrobial resistance and population struct...,Biology,biology1k_050522,1
41918,The INNO-LiPA Rif.,10.1128/JCM.41.9.4454-4456.2003,Journal of Clinical Microbiology,Direct Detection of Multidrug-Resistant Mycoba...,Biology,biology1k_050522,0
41919,Klebsiella pneumoniae is a pathogen responsibl...,10.1128/MRA.00024-20,Microbiology Resource Announcements,Genome Sequences of 12 Phages That Infect Kleb...,Biology,biology1k_050522,1


In [77]:
non_medicine_sources = ['acl_cambridge', 
                        
                        'predicts_acl_pm2500_vocabed', 'oct3_labels' "labels_oct7",'pm_CONCLUSIONS', "biology1k_050522", #adding these medicals because...
       'psych1k_100522', 'june152022_labels_envgeosoc',
       'june152022_labels_matscichem']

In [78]:
df = df[df["source"].isin(non_medicine_sources)]

In [79]:
df.source.value_counts()

pm_CONCLUSIONS                  5224
june152022_labels_envgeosoc     1597
biology1k_050522                1024
acl_cambridge                    996
june152022_labels_matscichem     487
psych1k_100522                   392
predicts_acl_pm2500_vocabed      222
Name: source, dtype: int64

In [80]:
df[df.labels == 0]

,text,doi,journalName,title,fields,source,labels
496,Obtained lexical entries are guaranteed to con...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
497,Sections 3 and 4 describe the features induced...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
498,"Until the system finds a multiple entry , it b...",NaN,NaN,NaN,Computer Science,acl_cambridge,0
499,-LRB- dictionary -RRB- No price for the new sh...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
500,Variation across languages can to a large exte...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
...,...,...,...,...,...,...,...
44381,The onset of a large number of a plant organ i...,10.1080/02286203.1993.11760199,International Journal of Modelling and Simulation,A model to estimate leaf-wetness duration usin...,Environmental Science,june152022_labels_envgeosoc,0
44383,Australia is frequently subject to droughts an...,10.1175/JCLI-D-13-00149.1,Journal of Climate,How Oceanic Oscillation Drives Soil Moisture V...,Environmental Science,june152022_labels_envgeosoc,0
44387,Strenuous effort on behalf of moral ideals was...,10.2979/trancharpeirsoc.52.1.06,Transactions of the Charles S. Peirce Society:...,The Geography of Strenuousness: “America” In W...,Sociology,june152022_labels_envgeosoc,0
44392,The area used for bioenergy crops (annual row ...,10.1186/S13705-015-0067-3,"Energy, Sustainability and Society",Legume-based mixed intercropping systems may l...,Environmental Science,june152022_labels_envgeosoc,0


In [81]:
df= df[["text", "labels"]]

In [82]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [83]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [84]:
df

,text,label
0,The difficulty with this task lies in the fact...,1
1,The problem with rich annotations is that they...,1
2,"As a consequence , when adapting existing meth...",1
3,The second problem of traditional word alignme...,1
4,The main drawback of these systems is that the...,1
...,...,...
44392,The area used for bioenergy crops (annual row ...,0
44393,Improper disposal of used polyethylene terepht...,1
44394,Airborne transmission of pathogenic aerosols v...,0
44395,Lead is one of the hazardous materials which i...,1


In [85]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [86]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/7954 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 7967 features created from 7954 samples.
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/996 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1988 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1988 features created from 1988 samples.


Running Evaluation:   0%|          | 0/249 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8221114124447522, 'tp': 570, 'tn': 1258, 'fp': 126, 'fn': 34, 'acc': 0.9195171026156942, 'eval_loss': 0.2172760886905049}


In [87]:
result

{'mcc': 0.8221114124447522,
 'tp': 570,
 'tn': 1258,
 'fp': 126,
 'fn': 34,
 'acc': 0.9195171026156942,
 'eval_loss': 0.2172760886905049}

In [88]:
import pandas as pd

In [89]:
dfx = pd.read_csv("downloads/16M_small_alldomains_clean.csv",  lineterminator='\n', error_bad_lines=False)

In [90]:
dfx[dfx["text"] == "nan"]

,text,doi,fields


In [96]:
df2 = pd.read_csv("downloads/train_final_june23_all.csv")


In [97]:
df2.fields.value_counts()

Medicine                 38453
Biology                   1024
Computer Science           996
Environmental Science      789
Psychology                 694
Sociology                  458
Geology                    350
Materials Science          185
Name: fields, dtype: int64

In [98]:
# dfx = dfx[dfx.fields.isin(['Environmental Science', 'Material Science', 'Psychology', 'Geology', 'Biology', 'Materials Science'])]
dfx = dfx[dfx.fields.isin(['Psychology', 'Materials Science', "Sociology", "Environmental Science"])]


In [99]:
dfx.fields.value_counts(normalize=True)

Materials Science        0.485300
Psychology               0.267998
Sociology                0.146912
Environmental Science    0.099790
Name: fields, dtype: float64

In [102]:
dfx["text"] = dfx["text"].astype(str)
len(dfx)


2717544

In [107]:
#p1 = "downloads/predicts_21June_5M_biomed.txt"
predicts_path = 'downloads/predicts_3M_non_medicine.txt'

In [108]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open(predicts_path, 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [ ]:
predicts = stepwise(50000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 50000


  0%|          | 0/50000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (750 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1232 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

  0%|          | 0/6290 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


2700000 2750000


  0%|          | 0/17544 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 17544 features created from 17544 samples.


  0%|          | 0/2205 [00:00<?, ?it/s]

In [ ]:
len(predicts)

2717544

In [ ]:
len([p for p in predicts if p==1])

445904

In [119]:
# with open(p1, 'r') as f:
#     p1 = f.read().split()#f.write("%s\n" % item)
                
# preds = p1 + p2
# len([p for p in preds if p == "1"])

In [ ]:
dfx["predicts"] = predicts

## dfx = dfx[:3000000]
dfx["predicts"] = preds

In [ ]:
dfx["predicts"] = dfx["predicts"].map(int)

In [ ]:
dfx = dfx[dfx["predicts"]==1]


In [ ]:
dfo = pd.read_csv("downloads/train_final_june23_all.csv")
dd = dfx[~dfx["doi"].isin(list(dfo["doi"]))]
dfx=dd

In [ ]:
dfx["fields"].value_counts()

Materials Science        188784
Psychology               109736
Environmental Science     96616
Sociology                 43088
Name: fields, dtype: int64

In [ ]:
len(dfx)

438224

In [120]:
#dr= dfx.iloc[::-1]
dfx.to_csv("downloads/June23_non_medicine.csv", index = False, header=True)

### Scraps

In [ ]:
df2 = dfx[dfx["fields"].isin(["Sociology", "Geology", "Environmental Science"])]
df2.to_csv("downloads/underrepresented_domains_predicts.csv", index = False, header=True)
df2 = dfx[dfx["fields"].isin(["Materials Science", "Psychology", "Chemistry"])]
df2.to_csv ("downloads/mid_represented_domains_predicts.csv", index = False, header=True)
#check that not in final110k_1605
dfx.to_csv("downloads/pos_preds_12062022.csv", index = False, header=True)
dfx[dfx["fieldsOfStudy"].isin(['Environmental Science', 'Sociology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

